In [1]:
import torch
import torch.nn as nn

import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))

from models._model import CBPLTrainer

In [2]:
config = {
    "peak_regions":"/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/test.chr1.adjusted.bed",
    "nonpeak_regions":"/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/test.chr1.negatives.adjusted.bed",
    "genome_fasta":"/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/chr1.fa",
    "cts_bw_file":"/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/test.chr1.bw",
    "negative_sampling_ratio":8,
    "train_size": 0.7,
    "batch_size": 32,
    "filters": 64,
    "n_dil_layers": 3,
    "conv1_kernel_size": 7,
    "profile_kernel_size": 5,
    "num_tasks": 1,
    "sequence_len": 1000,
    "out_pred_len": 1000,
    "learning_rate": 0.001,
    "max_epochs": 10
}

In [3]:
trainer =  CBPLTrainer(config)

Read in bed file of 6013 peaks
Read in bed file of 16900 peaks
Peak length = 1000    6013
Name: count, dtype: int64
Peak length = 1000    16900
Name: count, dtype: int64
Successfully loaded in data with 6013 positive and 16900 nonpeak regions!


In [14]:
trainer.dataset.seqs.shape

torch.Size([22913, 1000, 4])

In [16]:
for batch in trainer.train_dataloader:
    # `batch` will be a tuple, typically containing (inputs, targets)
    inputs, targets = batch
    break  # Exit the loop after getting the first batch


In [18]:
inputs.shape

torch.Size([32, 1000, 4])

In [19]:
targets.shape

torch.Size([32, 1000])

In [ ]:
trainer.dataset.seqs.shape

In [4]:
trainer.fit(epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2024-10-03 16:36:58.393360: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-03 16:36:58.393502: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

  | Name            | Type            | Params | Mode 
------------------------------------------------------------
0 | initial_conv    | CNNModule       | 1.9 K  | train
1 | dilated_convs   | ModuleList      | 37.1 K | train
2 | crop_layers     | ModuleList      | 0      | train
3 | profile_conv    | CNNModule       | 321    | train
4 | profile_crop    | Cropping1D      | 0      | train
5 | global_avg_pool | GlobalAvgPool1D | 0      | train
6 | count_dense     | Linear          | 65     | train

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: Given groups=1, weight of size [64, 4, 7], expected input[32, 1000, 4] to have 4 channels, but got 1000 channels instead

In [ ]:
model = BPNetLightning(
                filters = 5,
                n_dil_layers = 5,
                       conv1_kernel_size = 21, 
                 profile_kernel_size= 75,
                 num_tasks=1,
                 sequence_len=1000,
                 out_pred_len=1000,
                 learning_rate=1e-5)

In [ ]:
model

BPNetLightning(
  (initial_conv): CNNModule(
    (conv): Conv1d(4, 5, kernel_size=(21,), stride=(1,), padding=(10,))
    (activation): ReLU()
  )
  (dilated_convs): ModuleList(
    (0): DilatedConvModule(
      (conv): Conv1d(5, 5, kernel_size=(3,), stride=(1,), dilation=(2,))
      (activation): ReLU()
    )
    (1): DilatedConvModule(
      (conv): Conv1d(5, 5, kernel_size=(3,), stride=(1,), dilation=(4,))
      (activation): ReLU()
    )
    (2): DilatedConvModule(
      (conv): Conv1d(5, 5, kernel_size=(3,), stride=(1,), dilation=(8,))
      (activation): ReLU()
    )
    (3): DilatedConvModule(
      (conv): Conv1d(5, 5, kernel_size=(3,), stride=(1,), dilation=(16,))
      (activation): ReLU()
    )
    (4): DilatedConvModule(
      (conv): Conv1d(5, 5, kernel_size=(3,), stride=(1,), dilation=(32,))
      (activation): ReLU()
    )
  )
  (crop_layers): ModuleList(
    (0-4): 5 x Cropping1D()
  )
  (profile_conv): CNNModule(
    (conv): Conv1d(5, 1, kernel_size=(75,), stride=(1,), 

In [ ]:
dataset = ChromatinDataset(
    peak_regions="/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/test.chr1.adjusted.bed",
    nonpeak_regions="/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/test.chr1.negatives.adjusted.bed",
    genome_fasta="/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/chr1.fa",
    cts_bw_file="/gladstone/corces/lab/users/vishvak/chrombpnet_tutorial/own_data/test.chr1.bw",
    negative_sampling_ratio=0.8
)

Read in bed file of 6013 peaks
Read in bed file of 16900 peaks
Peak length = 1000    6013
Name: count, dtype: int64
Peak length = 1000    16900
Name: count, dtype: int64
Successfully loaded in data with 6013 positive and 4810 nonpeak regions!


In [ ]:
dataset.split(0.8,32)

(<torch.utils.data.dataloader.DataLoader at 0x7fba4632fb80>,
 <torch.utils.data.dataloader.DataLoader at 0x7fb5421f37f0>)

In [ ]:
class BPNetLightning(pl.LightningModule):
    def __init__(self, 
                 filters, 
                 n_dil_layers, 
                 conv1_kernel_size, 
                 profile_kernel_size, 
                 num_tasks, 
                 sequence_len, 
                 out_pred_len, 
                 learning_rate,
                 #counts_loss_weight,
                 #profile_loss_weight
                 )